In [ ]:
!pip install opencv-python mediapipe numpy

In [ ]:
!pip install --upgrade opencv-python mediapipe numpy

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# 각 라이브러리의 버전 확인
print(f"OpenCV version: {cv2.__version__}")
print(f"MediaPipe version: {mp.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# 데이터 스무딩 필터 적용[지수 이동 평균] (결과값 떨림 방지)
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# while 루프 시작 전
smooth_yaw = 0
smooth_pitch = 0

# Face Mesh 모델을 초기화합니다.
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:

    cap = cv2.VideoCapture('webcam.mp4')
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
        image = cv2.resize(image, (960, 540))
        # 여기서 이미지를 먼저 좌우 반전
        image = cv2.flip(image, 1)

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:

                
                # --- [1단계 시작] 고개 방향 추정 (최종 안정화 버전) ---
                # 이미지의 높이, 너비 가져오기
                img_h, img_w, _ = image.shape
                
                # 3D 얼굴 모델 좌표 (가장 표준적인 세트)
                # 이 좌표들은 실제 mm 단위가 아닌, 상대적인 비율입니다.
                face_3d_model = np.array([
                    [ 0.0,  0.0,   0.0],  # 코 끝 (Nose tip)
                    [ 0.0, -330.0, -65.0], # 턱 (Chin)
                    [-225.0,  170.0, -135.0], # 왼쪽 눈의 왼쪽 끝 (Left eye left corner)
                    [ 225.0,  170.0, -135.0], # 오른쪽 눈의 오른쪽 끝 (Right eye right corner)
                    [-150.0, -150.0, -125.0], # 왼쪽 입가 (Left Mouth corner)
                    [ 150.0, -150.0, -125.0]  # 오른쪽 입가 (Right mouth corner)
                ], dtype=np.float64)
                
                # 위 3D 모델에 해당하는 MediaPipe 랜드마크 인덱스
                # (순서가 매우 중요합니다!)
                landmark_idx = [1, 152, 263, 33, 291, 61]
                # 1: 코 끝, 152: 턱, 263: 왼쪽 눈 끝, 33: 오른쪽 눈 끝, 291: 왼쪽 입가, 61: 오른쪽 입가
                # (※ 주의: MediaPipe에서 좌/우는 보는 사람 기준이 아닌, 이미지 속 얼굴 기준입니다)
                
                # 2D 이미지 좌표 추출
                face_2d_points = np.zeros((6, 2), dtype=np.float64)
                for i, idx in enumerate(landmark_idx):
                    lm = face_landmarks.landmark[idx]
                    face_2d_points[i] = [lm.x * img_w, lm.y * img_h]
                
                # ★★★★★ 여기가 핵심 수정 부분 ★★★★★
                # 카메라 매트릭스 (Camera Matrix)
                # 초점 거리(focal length)를 이미지의 너비로 가정하는 것이 가장 일반적이고 안정적입니다.
                focal_length = img_w
                cam_matrix = np.array([
                    [focal_length, 0, img_w / 2],
                    [0, focal_length, img_h / 2],
                    [0, 0, 1]
                ])
                
                # 왜곡 계수 (Distortion Coefficients)는 없다고 가정합니다.
                dist_coeffs = np.zeros((4, 1), dtype=np.float64)
                
                # solvePnP 함수로 회전(rotation)과 변환(translation) 벡터를 계산
                success, rot_vec, trans_vec = cv2.solvePnP(
                    face_3d_model, face_2d_points, cam_matrix, dist_coeffs)
                
                # (시각화) 3D 축을 2D 이미지에 투영하여 그리기 (디버깅에 매우 유용)
                axis_points_3d = np.float32([[800, 0, 0], [0, 800, 0], [0, 0, 800]])
                axis_points_2d, _ = cv2.projectPoints(axis_points_3d, rot_vec, trans_vec, cam_matrix, dist_coeffs)
                
                # 코 끝 좌표를 정수형으로 변환
                nose_tip_2d = tuple(face_2d_points[0].astype(int))
                
                # 축 그리기 (빨간선: X축, 파란선: Z축)
                cv2.line(image, nose_tip_2d, tuple(axis_points_2d[0].ravel().astype(int)), (0, 0, 255), 3) # X축 (빨강)
                cv2.line(image, nose_tip_2d, tuple(axis_points_2d[1].ravel().astype(int)), (0, 255, 0), 3) # Y축 (초록)
                cv2.line(image, nose_tip_2d, tuple(axis_points_2d[2].ravel().astype(int)), (255, 0, 0), 3) # Z축 (파랑)
                
                # (안정적인 각도 계산) 회전 벡터를 회전 행렬로 변환
                rot_mat, _ = cv2.Rodrigues(rot_vec)
                
                # 회전 행렬로부터 오일러 각도 추출 (Gimbal Lock 문제 방지)
                sy = np.sqrt(rot_mat[0, 0] * rot_mat[0, 0] + rot_mat[1, 0] * rot_mat[1, 0])
                singular = sy < 1e-6
                if not singular:
                    x = np.arctan2(rot_mat[2, 1], rot_mat[2, 2])
                    y = np.arctan2(-rot_mat[2, 0], sy)
                    z = np.arctan2(rot_mat[1, 0], rot_mat[0, 0])
                else:
                    x = np.arctan2(-rot_mat[1, 2], rot_mat[1, 1])
                    y = np.arctan2(-rot_mat[2, 0], sy)
                    z = 0
                
                # 라디안을 각도로 변환 (180 / PI)
                pitch = -np.degrees(x)
                yaw = -np.degrees(y)
                roll = -np.degrees(z)

                # =============================================================
                # EMA 필터 적용 부분
                
                # 스무딩 강도 (0에 가까울수록 부드러움, 1에 가까울수록 빠름)
                alpha = 0.2
                
                # 첫 프레임에서는 초기값을 바로 사용
                if 'smooth_yaw' not in locals():
                    smooth_yaw = yaw
                    smooth_pitch = pitch
                else:
                    smooth_yaw = alpha * yaw + (1 - alpha) * smooth_yaw
                    smooth_pitch = alpha * pitch + (1 - alpha) * smooth_pitch
                # ==============================================================
                
                # 화면에 각도(Yaw, Pitch) 텍스트로 표시
                #cv2.putText(image, f"Yaw: {yaw:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                #cv2.putText(image, f"Pitch: {pitch:.2f}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                if smooth_yaw>=0:
                    cv2.putText(image, f"Yaw: {smooth_yaw:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(image, f"Yaw: {smooth_yaw:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                if smooth_pitch>=0:
                    cv2.putText(image, f"Pitch: {smooth_pitch:.2f}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(image, f"Pitch: {smooth_pitch:.2f}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                # --- [1단계 끝] ---

                
                # [수정된 부분] styles를 사용하지 않고 직접 DrawingSpec으로 스타일 지정
                # 얼굴 망 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=1, circle_radius=1))

                # 얼굴 외곽선 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
                
        cv2.imshow('MediaPipe Face Mesh', image)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# 칼만 필터 적용 (결과값 점프현상 방지)
import cv2
import mediapipe as mp
import numpy as np

# --------------------------------------------------------------------
# 1. 칼만 필터 및 상태 변수 초기화
# --------------------------------------------------------------------

def kalman_filter(kalman, measurement):

    # 예측 단계
    kalman['x'] = kalman['x']  # 상태 예측 (속도를 고려하지 않는 간단한 모델)
    kalman['p'] = kalman['p'] + kalman['q']  # 오차 공분산 예측

    # 업데이트 단계
    kalman['k'] = kalman['p'] / (kalman['p'] + kalman['r'])  # 칼만 이득 계산
    kalman['x'] = kalman['x'] + kalman['k'] * (measurement - kalman['x'])  # 추정값 업데이트
    kalman['p'] = (1 - kalman['k']) * kalman['p']  # 오차 공분산 업데이트
    
    return kalman['x']

# 칼만 필터의 상태를 저장하고 추적하기 위한 변수들을 초기화합니다.
# 이 변수들은 while 루프 밖에서 한번만 선언되어야 상태가 유지됩니다.
# - q (프로세스 노이즈): 모델이 얼마나 부정확한지에 대한 값. 작을수록 예측을 더 신뢰.
# - r (측정 노이즈): 센서(측정값)가 얼마나 부정확한지에 대한 값. 클수록 측정값을 덜 신뢰(스무딩 효과 강해짐).
kalman_yaw = {'q': 0.1, 'r': 25, 'x': 0, 'p': 0.1, 'k': 0}
kalman_pitch = {'q': 0.1, 'r': 25, 'x': 0, 'p': 0.1, 'k': 0}


# --------------------------------------------------------------------
# 2. MediaPipe 초기화 및 웹캠 설정
# --------------------------------------------------------------------

# MediaPipe의 드로잉 유틸리티와 Face Mesh 솔루션을 가져옵니다.
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Face Mesh 모델을 초기화합니다.
with mp_face_mesh.FaceMesh(
    max_num_faces=1,             # 최대 1개의 얼굴만 감지
    refine_landmarks=True,       # 눈, 입술 주변의 랜드마크를 더 정교하게 찾음
    min_detection_confidence=0.5, # 탐지 성공으로 간주할 최소 신뢰도
    min_tracking_confidence=0.5) as face_mesh: # 추적 성공으로 간주할 최소 신뢰도

    # 웹캠 비디오 캡처 객체를 생성합니다. (0은 기본 웹캠)
    cap = cv2.VideoCapture('webcam.mp4')

    # --------------------------------------------------------------------
    # 3. 메인 루프: 실시간 영상 처리
    # --------------------------------------------------------------------
    while cap.isOpened():
        # 웹캠에서 프레임을 하나씩 읽어옵니다.
        success, image = cap.read()
        if not success:
            print("웹캠 프레임을 읽을 수 없습니다.")
            break
        image = cv2.resize(image, (960, 540))
        # 거울 모드(좌우 반전)로 만들기 위해 이미지를 뒤집습니다.
        # 모든 좌표 계산과 그리기는 이 반전된 이미지를 기준으로 수행됩니다.
        image = cv2.flip(image, 1)

        # 성능 향상을 위해 이미지를 읽기 전용으로 설정하고, BGR을 RGB로 변환합니다.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # MediaPipe 모델로 이미지를 처리하여 얼굴 랜드마크를 검출합니다.
        results = face_mesh.process(image)

        # 화면에 그리기 위해 이미지를 다시 쓰기 가능하게 만들고, RGB를 BGR로 변환합니다.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 얼굴 랜드마크가 검출된 경우에만 아래 로직을 수행합니다.
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                
                # --------------------------------------------------------------------
                # 4. 고개 방향 추정 (Head Pose Estimation)
                # --------------------------------------------------------------------
                img_h, img_w, _ = image.shape

                # solvePnP에 사용할 3D 얼굴 모델 좌표 (상대적인 비율)
                face_3d_model = np.array([
                    [ 0.0,  0.0,   0.0],  # 코 끝
                    [ 0.0, -330.0, -65.0], # 턱
                    [-225.0,  170.0, -135.0], # 왼쪽 눈의 왼쪽 끝
                    [ 225.0,  170.0, -135.0], # 오른쪽 눈의 오른쪽 끝
                    [-150.0, -150.0, -125.0], # 왼쪽 입가
                    [ 150.0, -150.0, -125.0]  # 오른쪽 입가
                ], dtype=np.float64)

                # 위 3D 모델에 1:1로 매칭되는 MediaPipe 랜드마크 인덱스
                # 순서가 매우 중요합니다.
                landmark_idx = [1, 152, 263, 33, 291, 61]
                
                # 3D 모델에 매칭되는 2D 이미지 좌표를 추출합니다.
                face_2d_points = np.zeros((6, 2), dtype=np.float64)
                for i, idx in enumerate(landmark_idx):
                    lm = face_landmarks.landmark[idx]
                    face_2d_points[i] = [lm.x * img_w, lm.y * img_h]

                # 카메라의 내부 속성을 정의하는 카메라 매트릭스
                # 초점 거리를 이미지 너비로 가정하는 것이 일반적이고 안정적입니다.
                focal_length = img_w
                cam_matrix = np.array([
                    [focal_length, 0, img_w / 2],
                    [0, focal_length, img_h / 2],
                    [0, 0, 1]
                ])
                
                # 이미지 왜곡은 없다고 가정합니다.
                dist_coeffs = np.zeros((4, 1), dtype=np.float64)

                # 3D-2D 포인트 매칭을 통해 머리의 회전과 위치를 계산합니다.
                success, rot_vec, trans_vec = cv2.solvePnP(
                    face_3d_model, face_2d_points, cam_matrix, dist_coeffs)

                # 회전 벡터(rot_vec)를 사람이 이해하기 쉬운 오일러 각도(Yaw, Pitch, Roll)로 변환합니다.
                rot_mat, _ = cv2.Rodrigues(rot_vec)
                sy = np.sqrt(rot_mat[0, 0] * rot_mat[0, 0] + rot_mat[1, 0] * rot_mat[1, 0])
                singular = sy < 1e-6
                if not singular:
                    x = np.arctan2(rot_mat[2, 1], rot_mat[2, 2])
                    y = np.arctan2(-rot_mat[2, 0], sy)
                    z = np.arctan2(rot_mat[1, 0], rot_mat[0, 0])
                else:
                    x = np.arctan2(-rot_mat[1, 2], rot_mat[1, 1])
                    y = np.arctan2(-rot_mat[2, 0], sy)
                    z = 0
                
                # 라디안 단위를 각도(degree) 단위로 변환하고, 사용자에게 맞는 방향으로 부호를 조정합니다.
                pitch = -np.degrees(x)
                yaw = -np.degrees(y)
                roll = -np.degrees(z)

                # --------------------------------------------------------------------
                # 5. 필터링 및 시각화
                # --------------------------------------------------------------------

                # 계산된 raw Yaw/Pitch 값에 칼만 필터를 적용하여 노이즈를 제거하고 부드럽게 만듭니다.
                smooth_yaw = kalman_filter(kalman_yaw, yaw)
                smooth_pitch = kalman_filter(kalman_pitch, pitch)

                # 3D 축을 얼굴에 그려서 고개 방향을 시각적으로 보여줍니다.
                axis_points_3d = np.float32([[800, 0, 0], [0, 800, 0], [0, 0, 800]])
                axis_points_2d, _ = cv2.projectPoints(axis_points_3d, rot_vec, trans_vec, cam_matrix, dist_coeffs)
                nose_tip_2d = tuple(face_2d_points[0].astype(int))
                cv2.line(image, nose_tip_2d, tuple(axis_points_2d[0].ravel().astype(int)), (0, 0, 255), 3) # X축 (빨강)
                cv2.line(image, nose_tip_2d, tuple(axis_points_2d[1].ravel().astype(int)), (0, 255, 0), 3) # Y축 (초록)
                cv2.line(image, nose_tip_2d, tuple(axis_points_2d[2].ravel().astype(int)), (255, 0, 0), 3) # Z축 (파랑)

                # 최종 필터링된 값의 부호에 따라 텍스트 색상을 다르게 설정하여 시각적 피드백을 제공합니다.
                # Yaw 값 표시 (양수: 초록색, 음수: 빨간색)
                if smooth_yaw >= 0:
                    cv2.putText(image, f"Yaw: {smooth_yaw:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(image, f"Yaw: {smooth_yaw:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
                # Pitch 값 표시 (양수: 초록색, 음수: 빨간색)
                if smooth_pitch >= 0:
                    cv2.putText(image, f"Pitch: {smooth_pitch:.2f}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(image, f"Pitch: {smooth_pitch:.2f}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # 얼굴 망 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=1, circle_radius=1))

                # 얼굴 외곽선 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
        
        # 최종 결과 이미지를 'AI Interview Helper'라는 제목의 창에 보여줍니다.
        cv2.imshow('AI Interview Helper', image)

        # 'q' 키를 누르면 루프를 종료합니다.
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# --------------------------------------------------------------------
# 6. 리소스 해제
# --------------------------------------------------------------------
cap.release()
cv2.destroyAllWindows()